In [1]:
import pyodbc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pycaret.anomaly import *

In [2]:
## Conexion al DWH
cnxn = pyodbc.connect(
    driver='{SQL Server}',
    server='192.168.100.58',
    uid='bilectura',
    pwd='D1sp@p3l3s')
cursor = cnxn.cursor()
#cnxn.close()

In [3]:
codigo_SQL = "SELECT A.VTAANO AS 'Ano', A.VTAMES AS 'Mes', A.DSTCOD AS 'Codigo distrito', A.VTAZNA AS 'Codigo zona', A.VTANIT AS 'Nit cliente', A.VTAPRDCOD AS 'Codigo producto', B.TPOCOD AS 'Codigo tipo', B.LNACOD AS 'Codigo linea', B.GRPCOD AS 'Codigo grupo', B.CLDCOD AS 'Codigo calidad', SUM(A.VTAVLRVTA) AS 'Ventas' FROM V_VTA_VTAHEC A LEFT JOIN DIM_PRODUCTOS B ON A.PRMCOD = B.PRMCOD AND A.VTAPRDCOD = B.PRDCOD WHERE A.PRMCOD = 1 AND A.DSTCOD != 30 AND A.VTAZNA < 400 AND A.VTAFCH BETWEEN '2021-01-01' AND '2022-12-31' GROUP BY A.VTAANO, A.VTAMES, A.DSTCOD, A.VTAZNA, A.VTANIT, A.VTAPRDCOD, B.TPOCOD, B.LNACOD, B.GRPCOD, B.CLDCOD"

In [4]:
#Cargue de la data desde el ERP de Dispapeles y se guarda en df
cursor.execute(codigo_SQL)
rows = cursor.fetchall()
df = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

In [5]:
df

,Ano,Mes,Codigo distrito,Codigo zona,Nit cliente,Codigo producto,Codigo tipo,Codigo linea,Codigo grupo,Codigo calidad,Ventas
0,2022,11,10,10,80140914,27173,180,166,247,533,970200.0000
1,2021,11,90,20,28551571,1749,125,1,33,441,157624.0000
2,2021,10,90,21,30343334,1752,125,1,33,441,170544.0000
3,2022,12,20,2,800118202,838,201,20,1,60,157005.1000
4,2021,2,60,3,2080306,57284,210,112,66,220,73888.5000
...,...,...,...,...,...,...,...,...,...,...,...
525380,2022,11,40,70,800049074,10676,201,20,1,52,6080228.0000
525381,2022,7,40,64,901280334,55088,235,125,142,231,20929.4100
525382,2022,9,90,21,890701033,20326,235,120,180,330,782030.2000
525383,2021,10,20,63,900896300,55106,235,125,142,331,7000.0000


In [6]:
df2 = df.loc[(df["Codigo distrito"] == 10) & (df["Codigo zona"] == 18)]
df2

,Ano,Mes,Codigo distrito,Codigo zona,Nit cliente,Codigo producto,Codigo tipo,Codigo linea,Codigo grupo,Codigo calidad,Ventas
177,2022,6,10,18,901561242,27545,183,160,13,164,288529.2000
2127,2021,7,10,18,901276512,26390,183,160,13,164,577058.4000
4020,2021,5,10,18,901160593,26391,183,160,13,164,304558.6000
5066,2022,5,10,18,901267812,26393,183,160,13,164,298146.8400
5797,2022,9,10,18,900488618,27544,183,160,13,164,1094121.0000
7603,2022,5,10,18,901561242,27431,183,172,208,166,2595882.0000
8399,2021,10,10,18,901276512,26390,183,160,13,164,577058.4000
8665,2021,4,10,18,901270397,27672,180,196,204,74,484000.0000
10653,2021,11,10,18,901276512,26392,183,160,13,164,2596762.0000
11445,2022,6,10,18,900704122,27424,183,172,208,166,681858.0000


In [ ]:
exp_ano101 = setup(df, normalize = True, 
                   session_id = 123)

In [ ]:
iforest = create_model('iforest')

In [ ]:
print(iforest)

In [ ]:
svm = create_model('svm', fraction = 0.025)

In [ ]:
print(svm)

In [ ]:
models()

In [ ]:
iforest_results = assign_model(iforest)

In [ ]:
resultado = iforest_results[iforest_results["Anomaly"] == 1]
resultado = resultado[resultado["Anomaly_Score"] >= 0.04]
resultado

In [ ]:
plot_model(iforest)

In [ ]:
plot_model(iforest, plot = 'umap')

In [ ]:
iforest_results.to_csv("Anomalias detectadas.csv", encoding= 'utf-8', index= False)